### Import modules

In [ ]:
import pandas as pd
import numpy as np

import georges_core
from georges_core.sequences import PlacementSequence, SurveySequence, Element
from georges_core.units import ureg as _ureg
from georges_core import Kinematics

import georges
from georges import manzoni
from georges.manzoni import Input, Beam
from georges.manzoni.integrators import MadXIntegrator, \
                                        TransportFirstOrderTaylorIntegrator, \
                                        TransportSecondOrderTaylorIntegrator, \
                                        TransportFirstOrderTaylorIntegratorExact, \
                                        TransportSecondOrderTaylorIntegratorExact

### Define kinematics and beam

In [ ]:
particle = georges_core.particles.Proton
kin = georges_core.Kinematics(230*_ureg.MeV,
                              particle=particle, 
                              kinetic=True)

In [ ]:
beam_distr = georges_core.Distribution().from_5d_multigaussian_distribution(n=10000,
                                                                            XRMS=5e-3 *_ureg.m,
                                                                            PXRMS=5e-3 *_ureg.radians,
                                                                            YRMS=5e-3 *_ureg.m,
                                                                            PYRMS=5e-3*_ureg.radians).distribution
beam = Beam(kinematics = kin,
            distribution = beam_distr)

### Define sequence manually

In [ ]:
def define_sequence(integrator, aperture):
    
    d1 = Element.Drift(NAME="D1",
                   integrator=integrator,
                   L=0.3* _ureg.m,
                   APERTYPE="CIRCULAR",
                   APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    qf = Element.Quadrupole(NAME="Q1",
                            integrator=integrator,
                            L=0.3*_ureg.m,
                            K1=2*_ureg.m**-2,
                            APERTYPE="CIRCULAR",
                            APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    d2 = Element.Drift(NAME="D2",
                       integrator=integrator,
                       L=0.3*_ureg.m,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    b1 = Element.SBend(NAME="B1",
                       integrator=integrator,
                       L=1*_ureg.m,
                       ANGLE=30*_ureg.degrees,
                       K1=0*_ureg.m**-2,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    d3 = Element.Drift(NAME="D3",
                       integrator=integrator,
                       L=0.3*_ureg.m,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    qd = Element.Quadrupole(NAME="Q2",
                            integrator=integrator,
                            L=0.3*_ureg.m,
                            K1=-2*_ureg.m**-2,
                            APERTYPE="CIRCULAR",
                            APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    d4 = Element.Drift(NAME="D4",
                       integrator=integrator,
                       L=0.3*_ureg.m,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    b2 = Element.SBend(NAME="B2",
                       integrator=integrator,
                       L=1*_ureg.m,
                       ANGLE=-30*_ureg.degrees,
                       K1=0*_ureg.m**-2,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    d5 = Element.Drift(NAME="D5",
                       integrator=integrator,
                       L=0.3*_ureg.m,
                       APERTYPE="CIRCULAR",
                       APERTURE=[aperture*_ureg.cm, aperture*_ureg.cm])
    
    sequence = PlacementSequence(name="TEST")
    
    sequence.place(d1,at_entry=0)
    sequence.place_after_last(qf)
    sequence.place_after_last(d2)
    sequence.place_after_last(b1)
    sequence.place_after_last(d3)
    sequence.place_after_last(qd)
    sequence.place_after_last(d4)
    sequence.place_after_last(b2)
    sequence.place_after_last(d5)    

    return sequence

In [ ]:
sequence_madx = define_sequence(MadXIntegrator, 10)
sequence_transport_order_1 = define_sequence(TransportFirstOrderTaylorIntegrator, 10)
sequence_transport_order_2 = define_sequence(TransportSecondOrderTaylorIntegrator, 10)

### Load sequence using SurveySequence

In [ ]:
sequence_survey = SurveySequence(filename="survey.csv")
sequence_survey.expand(); # This must be done before tracking !

### Track with MANZONI

In [ ]:
mi_madx = manzoni.Input.from_sequence(sequence=sequence_madx)
mi_transport_order_1 = manzoni.Input.from_sequence(sequence=sequence_transport_order_1)
mi_transport_order_2 = manzoni.Input.from_sequence(sequence=sequence_transport_order_2)
mi_survey = manzoni.Input.from_sequence(sequence=sequence_survey)

obs_madx = georges.manzoni.SigmaObserver()
obs_transport_order_1 = georges.manzoni.SigmaObserver()
obs_transport_order_2 = georges.manzoni.SigmaObserver()
obs_survey = georges.manzoni.SigmaObserver()

mi_madx.track(beam=beam,
              observers=[obs_madx])
mi_transport_order_1.track(beam=beam,
              observers=[obs_transport_order_1])
mi_transport_order_2.track(beam=beam,
              observers=[obs_transport_order_2])
mi_survey.track(beam=beam,
              observers=[obs_survey])

res_madx = obs_madx.to_df()
res_transport_order_1 = obs_transport_order_1.to_df()
res_transport_order_2 = obs_transport_order_2.to_df()
res_survey = obs_survey.to_df()

res_survey.set_index("LABEL1",inplace=True)
res_survey.drop(index=['START','END'],inplace=True)

### Load BDSIM results

In [ ]:
bdsim_res = pd.read_csv('bdsim_results.csv')

### Plots for comparison

#### Generate missing columns for the dataframe

In [ ]:
df = sequence_madx.df.copy()
for line,_ in df.iterrows():
    if line != "D1":
        df.at[line,'AT_ENTRY'] = df.at[line,'AT_ENTRY'].m_as('m')
        df.at[line,'AT_CENTER'] = df.at[line,'AT_CENTER'].m_as('m')
        df.at[line,'AT_EXIT'] = df.at[line,'AT_EXIT'].m_as('m')
        
df.at['D1','AT_CENTER'] = df.at['D1','AT_CENTER'].m_as('m')
df.at['D1','AT_EXIT'] = df.at['D1','AT_EXIT'].m_as('m')

df['TYPE'] = ['Drift','Quadrupole','Drift','SBend','Drift','Quadrupole','Drift','SBend','Drift']
df['CLASS'] = ['Drift','Quadrupole','Drift','SBend','Drift','Quadrupole','Drift','SBend','Drift']
df['B'] = kin.brho * (df['ANGLE'] / df['L'] *_ureg.m)

#### Plot along X axis

In [ ]:
artist = georges_core.vis.PlotlyArtist(width=800)


artist.scatter(x=bdsim_res['S'],
               y=1e3*res_madx['BEAM_OUT_X'],
               marker={'color': 'blue', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="Manzoni - MadX",
               showlegend=True)

artist.scatter(x=bdsim_res['S'],
               y=1e3*res_transport_order_1['BEAM_OUT_X'],
               marker={'color': 'black', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="Manzoni - Transport 1",
               showlegend=True)

artist.scatter(x=bdsim_res['S'],
               y=1e3*res_transport_order_2['BEAM_OUT_X'],
               marker={'color': 'red', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="Manzoni - Transport 2",
               showlegend=True)

artist.scatter(x=bdsim_res['S'],
               y=1e3*res_survey['BEAM_OUT_X'],
               marker={'color': 'green', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="survey",
               showlegend=True)

artist.scatter(x=bdsim_res['S'],
               y=1e3*bdsim_res['X'],
               marker={'color': 'magenta', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="BDSIM",
               showlegend=True)

artist._layout = {'xaxis': {'title': 'S (m)'},
                  'yaxis': {'title': 'X size (mm)'}}
artist.plot_cartouche(df, vertical_position=1.15)
artist.render()

#### Plot along Y axis

In [ ]:
artist = georges_core.vis.PlotlyArtist(width=800)


artist.scatter(x=bdsim_res['S'],
               y=1e3*res_madx['BEAM_OUT_Y'],
               marker={'color': 'blue', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="Manzoni - MadX",
               showlegend=True)

artist.scatter(x=bdsim_res['S'],
               y=1e3*res_transport_order_1['BEAM_OUT_Y'],
               marker={'color': 'black', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="Manzoni - Transport 1",
               showlegend=True)

artist.scatter(x=bdsim_res['S'],
               y=1e3*res_transport_order_2['BEAM_OUT_Y'],
               marker={'color': 'red', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="Manzoni - Transport 2",
               showlegend=True)

artist.scatter(x=bdsim_res['S'],
               y=1e3*res_survey['BEAM_OUT_Y'],
               marker={'color': 'green', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="survey",
               showlegend=True)

artist.scatter(x=bdsim_res['S'],
               y=1e3*bdsim_res['Y'],
               marker={'color': 'magenta', 'symbol': 300, 'size': 6},
               line={'dash': 'dash'},
               mode='lines+markers',
               name="BDSIM",
               showlegend=True)

artist._layout = {'xaxis': {'title': 'S (m)'},
                  'yaxis': {'title': 'Y size (mm)'}}
artist.plot_cartouche(df, vertical_position=1.15)
artist.render()